In [ ]:
contents

In [ ]:
import py7zr
import pyarrow.parquet as pq
import pyarrow.csv as pc
import os
import glob
from datetime import datetime
from ftplib import FTP

ftp = FTP('ftp.mtps.gov.br')
ftp.login()
ftp.cwd('pdet/microdados/CAGED/')
dir_list = []
dir_list = ftp.nlst()[:]
#bike_share_file = 'CAGEDEST_012007.txt'
#parquet_file = 'CAGEDEST_012007.parquet'
#arrow_table = pc.read_csv(bike_share_file, parse_options=pc.ParseOptions(delimiter=";"),read_options=pc.ReadOptions(encoding='cp1252'))
#pq.write_table(arrow_table, parquet_file)
start_Geral = datetime.now()
for dir in dir_list:
    names_files = glob.glob("*.parquet")[:]
    try:
        ftp.cwd(dir)
    except:
        continue
    file_list=[]
    file_list = ftp.nlst()[:]
    for file in file_list:
        if any(os.path.splitext(file)[0]+".parquet" in n for n in names_files):
            continue
        if(os.path.splitext(file)[1]=='.7z'):
            try:
                start = datetime.now()  
                name_file=file[:-3]
                with open(file, 'wb') as fp:
                    ftp.retrbinary('RETR '+file, fp.write)
                with py7zr.SevenZipFile(file, mode='r') as z:
                    z.extractall()
                with py7zr.SevenZipFile(file, 'r') as zip:
                     allfiles = zip.getnames()
                share_file = allfiles[0]
                parquet_file = name_file[-4:]+name_file[-6:-4]+'.parquet'
                arrow_table = pc.read_csv(share_file, parse_options=pc.ParseOptions(delimiter=";"),read_options=pc.ReadOptions(encoding='cp1252'))
                pq.write_table(arrow_table, parquet_file)
                os.remove(name_file+'.txt')
                os.remove(name_file+'.7z')
                end = datetime.now()
                x = end - start
                print('O arquivo: ',name_file)
                print(f'Write process took {x} seconds.')
            except:
                continue
    ftp.cwd('..')
end = datetime.now()
x = end - start_Geral
print(f'Tempo Total {x} seconds.')

In [ ]:
name_file
name_file[-4:]
name_file[-6:-4]

In [9]:
## Gerar arquivo com os datatypes da colunas. 
import pandas as pd
import glob
import os
import pyarrow as pa
lprimeiro = True
names_files = glob.glob(".\parquet\*.parquet")[:]
for file in names_files:
    df = pd.read_parquet(file)
    df2 = df.dtypes.to_frame('dtypes').reset_index()
    print(type(df2['dtypes']))
    df2['dtypes'] = df2['dtypes'].apply(str)
    print(df2.head())
    df2 = df2.transpose()
    df2 = df2.reindex(sorted(df2.columns), axis=1)
    df2 = df2.rename(index={0: os.path.splitext(file)[0]})
    print(os.path.splitext(file)[0])
    if(lprimeiro):
        #dffinal=df2
        table=pa.Table.from_pandas(df)
        lprimeiro=False
        del df
        del df2
    else:
        new_table=pa.Table.from_pandas(df)
        updated_table = pa.concat_tables([table, new_table], promote=True)
        #dffinal= dffinal.append(df2)
        #del df
        del df2
    #parquet_file = name_file[-4:]+name_file[-6:-4]+'.parquet'
    #os.rename(r'C:\Users\Ron\Desktop\Test\Products.txt',r'C:\Users\Ron\Desktop\Test\Shipped Products.txt')
dffinal.to_csv('dtypes.csv')


ArrowMemoryError: malloc of size 453984576 failed

In [ ]:
for column in (dffinal.columns.to_list()):
    uniqueValues = dffinal[column].unique()
    if(len(uniqueValues)>1):
        print("Colunas com mais de um conteúdo: ",column)
        print(uniqueValues)


In [ ]:
import pyarrow.parquet as pq
import pyarrow.csv as pc
from datetime import datetime
start = datetime.now()
bike_share_file = 'CAGEDEST_012007.txt'
parquet_file = 'CAGEDEST_012007.parquet'
arrow_table = pc.read_csv(bike_share_file, parse_options=pc.ParseOptions(delimiter=";"),read_options=pc.ReadOptions(encoding='cp1252'))
pq.write_table(arrow_table, parquet_file)

end = datetime.now()
x = end - start
print(f'Write process took {x} seconds.')

In [ ]:
df = pd.read_parquet('CAGEDEST_012007.parquet')


In [ ]:
df

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

if __name__ == "__main__":
    sc = SparkContext(appName="CSV2Parquet")
    sqlContext = SQLContext(sc)
    
    schema = StructType([
            StructField("col1", IntegerType(), True),
            StructField("col2", IntegerType(), True),
            StructField("col3", StringType(), True),
            StructField("col4", StringType(), True),
            StructField("col5", StringType(), True),
            StructField("col6", DoubleType(), True)])
    
    rdd = sc.textFile("CAGEDEST_012007.txt").map(lambda line: line.split(";"))
    df = sqlContext.createDataFrame(rdd, schema)
    df.write.parquet('input-parquet')